In [2]:
import numpy as np
import pandas as pd

In [56]:
# load file
df1 = pd.read_csv('rating_m.csv')
df1 = df1.set_index(pd.to_datetime(df1['date'],format='%Y%m%d'))
df1['Date'] = pd.to_datetime(df1['date'],format='%Y%m%d')
df1 = df1.drop(['date'], axis=1)
df1 = df1.drop(['Unnamed: 0'], axis=1)
df1 = df1.dropna()
df1.head()

,rate,Ticker,r,Date
date,,,,
2010-01-31,B-,AAL,16.0,2010-01-31
2010-02-28,B-,AAL,16.0,2010-02-28
2010-03-31,B-,AAL,16.0,2010-03-31
2010-04-30,B-,AAL,16.0,2010-04-30
2010-05-31,B-,AAL,16.0,2010-05-31


In [5]:
#load file
df = pd.read_csv('z_lia_predictions.csv')
df = df.set_index(pd.to_datetime(df['date']))
df['Date'] = pd.to_datetime(df['date'])
df = df.drop(['date'], axis=1)
df = df.dropna()
df.head()

,date,Ticker,BS_TOT_LIAB2,zscore,new_lia,new_z
date,,,,,,
2010-03-31,2010-03-31,CVS,25590.0,1.443750,25590.0,1.443750
2010-06-30,2010-06-30,CVS,25652.0,1.375677,25652.0,1.375677
2010-09-30,2010-09-30,CVS,25076.0,1.405911,25076.0,1.405911
2010-12-31,2010-12-31,CVS,24435.0,1.507349,24435.0,1.507349
2011-03-31,2011-03-31,CVS,24876.0,1.499813,24876.0,1.499813


In [43]:
df.head()

,Date,Ticker,BS_TOT_LIAB2,zscore,new_lia,new_z
date,,,,,,
2010-03-31,2010-03-31,CVS,25590.0,1.443750,25590.0,1.443750
2010-06-30,2010-06-30,CVS,25652.0,1.375677,25652.0,1.375677
2010-09-30,2010-09-30,CVS,25076.0,1.405911,25076.0,1.405911
2010-12-31,2010-12-31,CVS,24435.0,1.507349,24435.0,1.507349
2011-03-31,2011-03-31,CVS,24876.0,1.499813,24876.0,1.499813


In [4]:
universe = list(set(df.Ticker.values))

In [52]:
data_zscore_lia = pd.DataFrame()
for ticker in universe:
    df00 = df[df.Ticker == ticker]
    lia_ori = df00[['BS_TOT_LIAB2']].resample('D').interpolate(method='cubic') 
    zscore_ori = df00[['zscore']].resample('D').interpolate(method='cubic') 
    zscore_new = df00[['new_z']].resample('D').interpolate(method='cubic') 
    lia_new = df00[['new_lia']].resample('D').interpolate(method='cubic') 
    ticker = df00[['Ticker']].resample('D', fill_method='ffill')
    
    daily_ZandLia = pd.concat([ticker, lia_ori,zscore_ori,zscore_new, lia_new], axis =1)
    
    data_zscore_lia = pd.concat([data_zscore_lia, daily_ZandLia])
    
    

D:\anacondasetup\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: fill_method is deprecated to .resample()
the new syntax is .resample(...).ffill()
  if __name__ == '__main__':


In [68]:
rating_daily = pd.DataFrame()
for ticker in universe:
    df01 = df1[df1.Ticker == ticker]
    ticker = df01[['Ticker']].resample('D', fill_method='ffill')
    rating = df01[['r']].resample('D', fill_method='ffill')
    
    concat = pd.concat([ticker, rating], axis =1)
    
    rating_daily = pd.concat([rating_daily, concat])
    

D:\anacondasetup\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: fill_method is deprecated to .resample()
the new syntax is .resample(...).ffill()
  after removing the cwd from sys.path.
D:\anacondasetup\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: fill_method is deprecated to .resample()
the new syntax is .resample(...).ffill()
  """


In [78]:
rating_daily['Date'] = rating_daily.index.values
data_zscore_lia['Date'] = data_zscore_lia.index.values

In [79]:
data = pd.merge(data_zscore_lia, rating_daily, on=('Ticker','Date'))

In [81]:
data.to_csv("lia_zscore_pred_daily.csv")

In [84]:
df2 = pd.read_csv('DATA_Company.csv')
# df2 = df2.set_index(pd.to_datetime(df2['date']))
df2['Date'] = pd.to_datetime(df2['date'])
df2 = df2.drop(['Unnamed: 0'], axis=1)
df2 = df2.drop(['date'], axis=1)
df2 = df2.drop(['EBIT','BS_TOT_ASSET','BS_TOT_LIAB2',
       'BS_PURE_RETAINED_EARNINGS','RSK_BB_ISSUER_DEFAULT','WORKING_CAPITAL','IS_SALES_AND_SERVICES_REVENUES'], axis=1)
df2 = df2.rename(columns = {'RSK_BB_IMPLIED_CDS_SPREAD':'CDS', 
                           'BB_1YR_DEFAULT_PROB':'df_BB',"equity":"Ticker"})
df2.head()

,CUR_MKT_CAP,CDS,df_BB,PX_LAST,Ticker,Date
0,58774.6991,16.0,0.000032,83.02,MMM,2010-01-04
1,58406.5608,15.0,0.000031,82.50,MMM,2010-01-05
2,59234.8720,15.0,0.000030,83.67,MMM,2010-01-06
3,59277.3495,15.0,0.000030,83.73,MMM,2010-01-07
4,59695.0450,15.0,0.000029,84.32,MMM,2010-01-08


In [85]:
data_all = pd.merge(data, df2, on=('Ticker','Date'))

In [90]:
data_all.to_csv("new_all_data.csv")